<a href="https://colab.research.google.com/github/8Raouf24/PFE/blob/master/create_ontol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install owlready2

     |████████████████████████████████| 20.8MB 1.4MB/s 
  Created wheel for owlready2: filename=Owlready2-0.24-cp36-cp36m-linux_x86_64.whl size=20263000 sha256=aac59f36a69787fa4be3a354fb754414ab68df152f70fe49185c142664c115db
  Stored in directory: /root/.cache/pip/wheels/66/87/0b/7a97bd19c9dc817b279aadb5dd920ce3a77050e2ab5a413f43
Successfully built owlready2


In [2]:
!pip install rdflib

     |████████████████████████████████| 235kB 2.7MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 


In [3]:
from owlready2 import *
import gzip
import json
import rdflib


In [4]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Musical_Instruments_5.json.gz

--2020-08-18 20:53:50--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Musical_Instruments_5.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39356138 (38M) [application/octet-stream]
Saving to: ‘Musical_Instruments_5.json.gz’

Musical_Instruments 100%[===================>]  37.53M  14.9MB/s    in 2.5s    

2020-08-18 20:53:53 (14.9 MB/s) - ‘Musical_Instruments_5.json.gz’ saved [39356138/39356138]



In [5]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_Musical_Instruments.json.gz

--2020-08-18 20:53:55--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_Musical_Instruments.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170266128 (162M) [application/octet-stream]
Saving to: ‘meta_Musical_Instruments.json.gz’

meta_Musical_Instru 100%[===================>] 162.38M  21.9MB/s    in 8.2s    

2020-08-18 20:54:03 (19.9 MB/s) - ‘meta_Musical_Instruments.json.gz’ saved [170266128/170266128]



In [6]:
onto = get_ontology("http://www.semanticweb.org//Amazon_Ontol#")
ns = "http/AmazonOntol.org/nn.owl#"
with onto:
  class Article(Thing):
    pass

  class User(Thing):
    pass

  class Category(Thing):
    pass
  
  class Review(Thing):
    pass

  class Brand(Thing):
    pass

  class Main_Category(Category):
    pass

  class Other_Category(Category):
    pass

  AllDisjoint([Main_Category,Other_Category])

  class Asin(DataProperty,FunctionalProperty):
    domain = [Article]
    range = [str]

  class Title(DataProperty,FunctionalProperty):
    domain = [Article]
    range = [str]

  class Feature(Article >> str):
    pass
  
  class Description(DataProperty,FunctionalProperty):
    domain = [Article]
    range = [str]

  class Price(DataProperty,FunctionalProperty):
    domain = [Article]
    range = [float]

  class fromCategory(Article >> Category):
    pass

  class fromBrand(Article >> Brand):
    pass

  class alsoViewed(Article >> Article):
    pass

  class alsoBought(Article >> Article):
    pass

  class reviewerID(DataProperty, FunctionalProperty):
    domain = [User]
    range = [str]

  class vote(DataProperty,FunctionalProperty):
    domain = [Review]
    range = [int]

  class reviewText(DataProperty,FunctionalProperty):
    domain = [Review]
    range = [str]

  class Name(DataProperty,FunctionalProperty):
    domain = [Thing]
    range = [str]

In [7]:
onto.save("sortie.owl")

In [8]:
import gzip
import json
import pandas as pd

In [9]:

data = []
with gzip.open('Musical_Instruments_5.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))

In [10]:
df = pd.DataFrame.from_dict(data)

data_dict = {'userID':df['reviewerID'],'itemID':df['asin'],'rating':df['overall']}
df_rs = pd.DataFrame.from_dict(data_dict)

In [11]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        d = json.loads(l)
        yield d
corpus = parse('meta_Musical_Instruments.json.gz')

In [12]:
df = pd.DataFrame.from_dict(data)

In [13]:
df.isna().sum()

overall                0
verified               0
reviewTime             0
reviewerID             0
asin                   0
reviewerName          25
reviewText            48
summary               51
unixReviewTime         0
vote              196615
style             110082
image             227503
dtype: int64

In [14]:
df.drop_duplicates("asin","last")

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
37,5.0,False,"10 27, 2011",A19KOCD0UWK736,0786615206,Doc JD,No more pain finally! I had a foot stool/stand...,A-Frame guitar support should be used by ALL c...,1319673600,3,NaN,NaN
71,3.0,True,"07 24, 2012",A1K1MNVOJVP024,1480360295,Lurchie,I'm slowly teaching myself to play guitar and ...,Kinda like Guitar Hero for your instrument,1343088000,5,NaN,NaN
107,5.0,True,"05 8, 2008",A2X85K6IXO0LX5,B00001W0DT,Soundchaser88,I'm a professional piano technician and part-t...,Excellent Value!,1210204800,7,NaN,NaN
110,3.0,True,"03 27, 2004",A2B1OJ22IYM3NR,B00001W0DH,K. Lyman,These headphones are very clean. Highs are cr...,"Sounds good, but may not be what you want",1080345600,2,NaN,NaN
155,1.0,False,"04 19, 2005",A32QOZWIKQDNZX,B00004UE2D,Kindle Customer,If you read this it means you in the same poin...,How to buy classical guitar.,1113868800,43,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
231370,1.0,True,"06 24, 2018",A2V2WSB5AVI4N7,B01HGLUP14,Summit,GARBAGE. One doesn't work right out of the pac...,Garbage.,1529798400,NaN,"{'Size:': ' Adapter', 'Color:': ' Male to 2 Fe...",NaN
231373,5.0,True,"10 11, 2016",A20P189TK58PBR,B01HHJXKRG,Nico,It does what it says it does. Pretty much you ...,Perfectly described,1476144000,2,{'Color:': ' Assorted Colors'},NaN
231381,5.0,True,"08 20, 2017",A25ORF72RXGY3F,B01HI1VM28,Henry King,A good little tool....,Great tool,1503187200,NaN,NaN,NaN
231385,5.0,True,"05 25, 2018",A3ML30U8VI68L,B01HI5M4GW,C Valero,ok,Five Stars,1527206400,NaN,NaN,NaN


In [15]:
onto = onto = get_ontology("sortie.owl").load()

In [16]:
list_price = []
with onto:

    obj = next(corpus)
    #On initialise la main category vu que ce sera la meme pour tout les articles du fichiers
    class_Mcat = onto.Main_Category
    Mcat = class_Mcat()
    Mcat.iri = ns + obj['main_cat'].replace(' ','_')

    for i in range(1000):
        
        if onto.search(iri="*" + obj['asin']) == []:
            #Je cree mon objet article
            class_article = onto.Article
            article = class_article()
            article.iri = ns + obj['asin']
            article.Asin = obj['asin']
            article.Title = obj['title']

            #Je verifie l'existence de la description , si oui je l'ajoute
            if 'description' in obj :
                article.Description = str(obj['description'])
            #Je verifie l'existance du prix , en son abscence je remplacerai par "unknown price " afin d'utiliser le web scrapper
            if 'price' in obj:
                if obj['price'] != "":
                    article.price = obj['price']
                    list_price.append(obj['price'])
                else:
                    article.price = "Unknown Price"
            else:
                article.price = "Unknown Price"

            #Verification de l'existance de la marque , si elle presente dans le fichier json , je verifie si elle est dans mon ontologie , si non , je la cree , si oui je la link avec l'objet
            if 'brand' in obj:
                if obj['brand'] != '':
                    if onto.search(iri="*" + obj['brand'].replace(' ','_')) == []:
                        class_brand = onto.Brand
                        brand = class_brand()
                        brand.iri = ns + "brand"+obj['brand'].replace(' ','_').replace('\"','')
                        brand.Name = obj['brand']
                        article.fromBrand.append(brand)
                    else :
                        article.fromBrand.append(onto.search(iri="*" + obj['brand'].replace(' ','_'))[0])


            #Meme travail pour les categories , sauf qu'ici les categories sont sous formes de liste , en plus de ca il y'a une erreur dans notre data set ou la description se retrouve souvent dans le champ categorie apres un chaine vide
            if 'category' in obj:
                if obj['category'] != []:
                    if '' in obj['category']:
                        limite = obj['category'].index('')
                        for i in range(1,limite):
                            if onto.search(iri="*" + obj['category'][i].replace(' ','_')) == []   :
                                if len(obj['category'][i].split(' '))<8 :
                                    class_cat = onto.Other_Category
                                    cat = class_cat()
                                    cat.iri = ns + obj['category'][i].replace(' ','_')
                                    cat.Name = obj['category'][i]
                                    article.fromCategory.append(cat)
                            else :
                                article.fromCategory.append(cat)
                    else:
                        for i in obj['category']:
                            if onto.search(iri="*" + i.replace(' ','_')) == [] :
                                if len(i.split(' ')) < 8:
                                    class_cat = onto.Other_Category
                                    cat = class_cat()
                                    cat.iri = ns + i.replace(' ','_')
                                    cat.Name = i
                                    article.fromCategory.append(cat)
                            else:
                                article.fromCategory.append(onto.search(iri="*" + i.replace(' ','_'))[0])


        obj=next(corpus)
    

    


  

In [17]:
onto.save("sortiefinal.owl", format="ntriples")

In [20]:
!cat sortiefinal.owl


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
<http/AmazonOntol.org/nn.owl#Singing_Bowls> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#NamedIndividual> .
<http/AmazonOntol.org/nn.owl#Singing_Bowls> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.semanticweb.org//Amazon_Ontol#Other_Category> .
<http/AmazonOntol.org/nn.owl#B000241NRS> <http://www.semanticweb.org//Amazon_Ontol#fromCategory> <http/AmazonOntol.org/nn.owl#Singing_Bowls> .
<http/AmazonOntol.org/nn.owl#B000246SKU> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#NamedIndividual> .
<http/AmazonOntol.org/nn.owl#B000246SKU> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.semanticweb.org//Amazon_Ontol#Article> .
<http/AmazonOntol.org/nn.owl#B000246SKU> <http://www.semanticweb.org//Amazon_Ontol#fromBrand> <http/AmazonOntol.org/nn.owl#brandSony> .
<http/AmazonOntol.org/nn.owl#B000246SKU> <http://www.semanti

In [18]:
graph = rdflib.Graph()
graph.parse("sortiefinal.owl",format="turtle")
open("sortieturtle.rdf","w")
graph.serialize("sortieturtle.rdf",format="turtle")


file:///content/http/AmazonOntol.org/nn.owl#</span does not look like a valid URI, trying to serialize this will break.


BadSyntax: ignored

In [19]:
!cat sortieturtle.rdf

cat: sortieturtle.rdf: No such file or directory


In [ ]:
requete = """ 
prefix ns1: <http://www.semanticweb.org//Amazon_Ontol#> 
prefix ns2: <http://www.w3.org/2002/07/owl#> 
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
prefix xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?i ?d
WHERE{
?i rdf:type ns1:Article .
?i ns1:Description ?d .
} 
"""
result = graph.query(requete)
for i in result:
        print(i)

In [ ]:
final = list(result)

In [ ]:
dico = {'item':[],'description':[]}
for i in final :
  dico['item'].append(i[0].split('#')[-1])
  dico['description'].append(i[1])


In [ ]:
df = pd.DataFrame.from_dict(dico)

In [ ]:
frames = [df,df_rs]
pd.concat(frames,join='iner')

In [ ]:
requete = """ 
prefix ns1: <http://www.semanticweb.org//Amazon_Ontol#> 
prefix ns2: <http://www.w3.org/2002/07/owl#> 
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
prefix xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?i ?t
WHERE{
?i rdf:type ?t .
} 
"""
result = graph.query(requete)
for i in result:
        print(i)